In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_764536/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [5]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_2_9_19,0.997451,0.811648,0.975350,0.997652,0.995259,0.017045,1.259511,0.041886,0.032539,0.037212,0.205926,0.130556,1.001359,0.136114,146.143824,230.246256,"Hidden Size=[17], regularizer=0.01, learning_r..."
1,model_2_9_18,0.997449,0.811478,0.976704,0.997784,0.995522,0.017060,1.260649,0.039586,0.030707,0.035146,0.215596,0.130613,1.001361,0.136173,146.142077,230.244509,"Hidden Size=[17], regularizer=0.01, learning_r..."
2,model_2_9_20,0.997444,0.811800,0.974033,0.997523,0.995002,0.017091,1.258496,0.044125,0.034323,0.039224,0.196844,0.130733,1.001363,0.136299,146.138388,230.240820,"Hidden Size=[17], regularizer=0.01, learning_r..."
3,model_2_9_17,0.997436,0.811287,0.978090,0.997919,0.995791,0.017148,1.261923,0.037230,0.028834,0.033032,0.225887,0.130949,1.001368,0.136524,146.131784,230.234216,"Hidden Size=[17], regularizer=0.01, learning_r..."
4,model_2_9_21,0.997430,0.811935,0.972753,0.997398,0.994753,0.017189,1.257591,0.046299,0.036059,0.041179,0.188318,0.131106,1.001371,0.136688,146.126989,230.229421,"Hidden Size=[17], regularizer=0.01, learning_r..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,model_14_7_15,0.993771,0.832877,0.985221,0.992243,0.991231,0.041652,1.117551,0.057944,0.179771,0.118858,0.251737,0.204089,1.002623,0.212777,168.356805,267.085746,"Hidden Size=[20], regularizer=0.01, learning_r..."
172,model_14_8_0,0.993755,0.832649,0.947420,0.998599,0.998516,0.041763,1.119076,0.009539,0.014453,0.011996,0.260658,0.204361,1.002630,0.213061,168.351475,267.080417,"Hidden Size=[20], regularizer=0.01, learning_r..."
177,model_14_7_5,0.993723,0.830567,0.990441,0.993656,0.993194,0.041974,1.133001,0.037476,0.147012,0.092244,0.269420,0.204876,1.002643,0.213598,168.341398,267.070340,"Hidden Size=[20], regularizer=0.01, learning_r..."
179,model_14_7_16,0.993699,0.832968,0.984467,0.992127,0.991023,0.042132,1.116942,0.060901,0.182445,0.121673,0.250493,0.205261,1.002653,0.213999,168.333890,267.062832,"Hidden Size=[20], regularizer=0.01, learning_r..."
